# Word2Vec (Skipgram )

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
import nltk
from nltk.corpus import semcor

nltk.download('semcor')

In [ ]:
torch.cuda.is_available(), torch.cuda.get_device_name(0)

In [ ]:
semcor.words()

## 1. Load data

In [5]:
#1. tokenization
corpus = semcor.sents()

In [6]:
#2. numeralization
#find unique words
flatten = lambda l: [item for sublist in l for item in sublist]
#assign unique integer
vocabs = list(set(flatten(corpus))) #all the words we have in the system - <UNK>

In [7]:
#create handy mapping between integer and word
word2index = {v:idx for idx, v in enumerate(vocabs)}


In [8]:
vocabs.append('<UNK>')
word2index['<UNK>'] = len(vocabs) - 1  

In [9]:
index2word = {v:k for k, v in word2index.items()}

## 2. Prepare train data

In [10]:
#create pairs of center word, and outside word

def random_batch(batch_size, corpus, window_size=2):

    skipgrams = []

    #loop each corpus
    for doc in corpus:
        #look from the 2nd word until second last word
        for i in range(window_size, len(doc)-window_size):
            #center word
            center = word2index[doc[i]]
            
            #outside words (window size on both sides of the center word)
            outside_start =  i - window_size
            outside_end =  i + window_size + 1 # +1 because the end index is exclusive

            # outside = []
            # loop through the outside words, append to the list 'outside'
            for j in range(outside_start, outside_end):
                if i != j:  # Skip the center word
                    outside= word2index[doc[j]]
                    skipgrams.append([center, outside])

                
    random_index = np.random.choice(range(len(skipgrams)), batch_size, replace=False)
    
    inputs, labels = [], []
    for index in random_index:
        inputs.append([skipgrams[index][0]])
        labels.append([skipgrams[index][1]])
        
    return np.array(inputs), np.array(labels)
            
x, y = random_batch(2, corpus)

In [ ]:
x.shape  #batch_size, 1

In [ ]:
x

In [ ]:
y.shape  #batch_size 1

## 3. Model

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

In [14]:
class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center, outside, all_vocabs):
        center_embedding     = self.embedding_center(center)  #(batch_size, 1, emb_size)
        outside_embedding    = self.embedding_center(outside) #(batch_size, 1, emb_size)
        all_vocabs_embedding = self.embedding_center(all_vocabs) #(batch_size, voc_size, emb_size)
        
        top_term = torch.exp(outside_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2))
        #batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) = (batch_size, 1) 

        lower_term = all_vocabs_embedding.bmm(center_embedding.transpose(1, 2)).squeeze(2)
        #batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size) 
        
        lower_term_sum = torch.sum(torch.exp(lower_term), 1)  #(batch_size, 1)
        
        loss = -torch.mean(torch.log(top_term / lower_term_sum))  #scalar
        
        return loss
        

In [ ]:
#prepare all vocabs

batch_size = 2
voc_size   = len(vocabs)

def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs

In [ ]:
model = Skipgram(voc_size, 2)
model

In [17]:
input_tensor = torch.LongTensor(x)
label_tensor = torch.LongTensor(y)

In [18]:
loss = model(input_tensor, label_tensor, all_vocabs)

In [ ]:
loss

## 4. Training

In [20]:
batch_size = 2
emb_size   = 2
model      = Skipgram(voc_size, emb_size)
optimizer  = optim.Adam(model.parameters(), lr=0.001)

In [21]:
def log_epoch(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time

start = time.time()
num_epochs = 100
window_size = 5

for epoch in range(num_epochs):
    
    #get batch
    input_batch, label_batch = random_batch(batch_size, corpus, window_size)
    input_tensor = torch.LongTensor(input_batch)
    label_tensor = torch.LongTensor(label_batch)
    
    #predict
    loss = model(input_tensor, label_tensor, all_vocabs)
    
    #backprogate
    optimizer.zero_grad()
    loss.backward()
    
    #update alpha
    optimizer.step()
    # Log epoch time
    epoch_mins, epoch_secs = log_epoch(start, time.time())
    
    #print the loss
    if (epoch + 1) % 10 == 0:
        print(f"Epoch: {epoch + 1} | Loss: {loss:.6f} | Time: {epoch_mins}m {epoch_secs}s")

## 5. Plot the embeddings

Is fruit really near to banana?
Is fruit really far from cat?

In [ ]:
vocabs

In [ ]:
banana = torch.LongTensor([word2index['banana']])
banana

In [ ]:
banana_embed_c = model.embedding_center(banana)
banana_embed_o = model.embedding_outside(banana)
banana_embed   = (banana_embed_c + banana_embed_o) / 2
banana_embed

In [ ]:
banana_embed_o

In [29]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    
    return embed[0][0].item(), embed[0][1].item()

In [ ]:
get_embed('fruit')

In [ ]:
get_embed('cat')

In [ ]:
get_embed('dog')

In [ ]:
get_embed('banana')

## 6. Cosine similarity

In [ ]:
banana = get_embed('banana')
banana

In [ ]:
fruit = get_embed('fruit')
fruit

In [ ]:
unk = get_embed('<UNK>')
unk

In [ ]:
np.array(banana) @ np.array(unk)

In [ ]:
#more formally is to divide by its norm
def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

print(cosine_similarity(np.array(banana), np.array(unk)))
print(cosine_similarity(np.array(banana), np.array(fruit)))

## Export Model

In [4]:
import pickle

In [ ]:
filename = 'skipgram_model.pkl'
pickle.dump(model, open(filename, 'wb'))

## Export word2index and index2word

In [ ]:
pickle.dump(word2index, open('skipgram_word2index.pkl', 'wb'))
pickle.dump(index2word, open('skipgram_index2word.pkl', 'wb'))